In [78]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
inputFile = 'shakespeare.txt'
with open(inputFile) as f:
  text = f.read()
  print('length of dataset in characters: ', len(text))

length of dataset in characters:  1115393


In [4]:
# this is the vocabulary i.e. all the unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [5]:
# create a mapping from characters to integers (i.e. encoding and decoding)
# this is needed as the tensors work with numbers

# create two lookup dictionaries from char to int, and vice versa
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

test = encode('Hello World!')
print(test)
print(decode(test))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42, 2]
Hello World!


In [6]:
# encode the whole input text and store it as a tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)

In [7]:
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
# split data into training and valdiation sets
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
block_size = 8
train_data[:block_size + 1] # we need +1 to get 8 training sets
 

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size] # the training input
y = train_data[1:block_size + 1] # the resulting test (one offset compared to train)
for t in range(block_size):
  context = x[:t + 1]
  target = y[t]
  print(f'when input is {context} the target is: {target}')

when input is tensor([18]) the target is: 47
when input is tensor([18, 47]) the target is: 56
when input is tensor([18, 47, 56]) the target is: 57
when input is tensor([18, 47, 56, 57]) the target is: 58
when input is tensor([18, 47, 56, 57, 58]) the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [10]:
torch.manual_seed(13371337)
batch_count = 4  # how many sequences will be processed in time i.e. how much training can we do in parallell afaik
block_size = 8 # how large is the context i.e. how many chars can we look back at and use in our prediction

def get_batches(data, block_size, batch_count):
  ix = torch.randint(len(data) - block_size, (batch_count,)) 
  x = torch.stack([data[i:i + block_size] for i in ix]) # training input
  y = torch.stack([data[i + 1:i+block_size + 1] for i in ix]) # target output
  return x, y

xb, yb = get_batches(train_data, block_size, batch_count)
print('inputs: ')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('-------')

for b in range(batch_count):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f'for context {context.tolist()} the target is {target}')


inputs: 
torch.Size([4, 8])
tensor([[46, 39, 58,  1, 51, 47, 57, 41],
        [39, 47, 42,  0, 20, 47, 57,  1],
        [ 1, 42, 43, 39, 58, 46,  6,  1],
        [59, 50, 58, 12,  0, 21, 58,  1]])
targets:
torch.Size([4, 8])
tensor([[39, 58,  1, 51, 47, 57, 41, 39],
        [47, 42,  0, 20, 47, 57,  1, 58],
        [42, 43, 39, 58, 46,  6,  1, 39],
        [50, 58, 12,  0, 21, 58,  1, 52]])
-------
for context [46] the target is 39
for context [46, 39] the target is 58
for context [46, 39, 58] the target is 1
for context [46, 39, 58, 1] the target is 51
for context [46, 39, 58, 1, 51] the target is 47
for context [46, 39, 58, 1, 51, 47] the target is 57
for context [46, 39, 58, 1, 51, 47, 57] the target is 41
for context [46, 39, 58, 1, 51, 47, 57, 41] the target is 39
for context [39] the target is 47
for context [39, 47] the target is 42
for context [39, 47, 42] the target is 0
for context [39, 47, 42, 0] the target is 20
for context [39, 47, 42, 0, 20] the target is 47
for context [

In [25]:
def generateFromNewLine(model, num_new_tokens):
  newline = torch.zeros((1, 1), dtype=torch.long)
  output = model.generate(idx = newline, max_new_tokens=num_new_tokens)
  print(decode(output[0].tolist()))

In [28]:
#create the bigram language model
import torch
import torch.nn as nn
from torch.nn import functional as F
import math

torch.manual_seed(13371337)


class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()

    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, contexts, targets=None):

    # we make a prediction based on a single token
    logits = self.token_embedding_table(contexts) # Batch Time Channel tensor (BTC)

    if targets is None:
      return logits, None
    
    # we have provided targets so compute the loss
    # we need to reshape the data for use in cross entropy
    B, T, C = logits.shape
    logits = logits.view(B * T, C)
    targets = targets.view(B * T)

    # compare to the target using neg log likeliehood loss
    loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of incdices into the current context
    for _ in range(max_new_tokens):
      logits, loss = self.forward(idx)
      logits = logits[:, -1, :] # pick the last element from the time dimension B, _T_, C
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1) # samples from the probabilities
      idx = torch.cat((idx, idx_next), dim=1) # add the prediction to the time dimension
    return idx

m = BigramLanguageModel(vocab_size)
out, loss = m.forward(xb, yb)
print(out.shape)
print("expected loss", -math.log(1/vocab_size))
print(loss)

generateFromNewLine(m, 250)

torch.Size([32, 65])
expected loss 4.174387269895637
tensor(4.7554, grad_fn=<NllLossBackward0>)

PB
s'bLVkUtSQo&Fbn:b;U
tDB 3g.Ls?F:Iu'vLeAmXBrL$J?InfhGrh,f3L$ l:skEO,3bHsN.b'XXnkSSS'b&XKwThvv?;ffSgPs.upzWQ FU;XChmK!RFDKrFQpS:H!oLewTQ;&xCL JZWQrFRqgInfnZ3-jgz..jgAbwpeAjm,:zbVRFfcW?IkAF XBtlkBfljPa
CzTAaos'xKyBFUAkKVtQ$ brkw$BmBmTnQtynS'CG,cd.Mt 


In [29]:
@torch.no_grad()
def estimate_loss(model, data, block_size, batch_count, eval_iters):
  out = {}
  model.eval()
  losses = torch.zeros(eval_iters)
  for k in range(eval_iters):
    X, Y = get_batches(data, block_size, batch_count)
    logits, loss = model.forward(X, Y)
    losses[k] = loss.item()
  model.train() # why this one?
  return losses.mean()
      

In [21]:
def train_model(model, train_data, val_data, max_iters):
  optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
  batch_size = 32
  eval_iters = 200
  eval_interval = 200
  for steps in range(max_iters):
    xb, yb = get_batches(train_data, block_size, batch_size)
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if steps % eval_interval == 0:
      train_loss = estimate_loss(model, train_data, block_size, batch_size, eval_iters)
      val_loss = estimate_loss(model, val_data, block_size, batch_size, eval_iters)
      print(f"step {steps}: train loss {train_loss:.4f}, val loss {val_loss:.4f}")



In [30]:
train_model(m, train_data, val_data, 3500)

step 0: train loss 4.7470, val loss 4.7720
step 200: train loss 4.5140, val loss 4.5406
step 400: train loss 4.3034, val loss 4.3282
step 600: train loss 4.0961, val loss 4.1307
step 800: train loss 3.9225, val loss 3.9445
step 1000: train loss 3.7580, val loss 3.7775
step 1200: train loss 3.6149, val loss 3.6365
step 1400: train loss 3.4655, val loss 3.5047
step 1600: train loss 3.3446, val loss 3.3821
step 1800: train loss 3.2391, val loss 3.2717
step 2000: train loss 3.1540, val loss 3.1771
step 2200: train loss 3.0643, val loss 3.0943
step 2400: train loss 2.9880, val loss 3.0194
step 2600: train loss 2.9280, val loss 2.9461
step 2800: train loss 2.8676, val loss 2.8924
step 3000: train loss 2.8269, val loss 2.8443
step 3200: train loss 2.7841, val loss 2.8031
step 3400: train loss 2.7382, val loss 2.7542


In [31]:
generateFromNewLine(m, 250)


WIut?YO MNaIV:
r, ly,-uesUjENu:me C
WjeleP thHANAchenges,:elly b myTly p&:Znalyof mo!-ONunBCHd thid os-:s wise Ijesum foo d
mifsectLLxVy y tZd trorS
KE H'j!:
Ifenkis FST;sit oL3orAscN,XGokipe&NGu
And pamerngell;

HNas I Iker se
Inorcr athytr!-Hure, L


In [16]:
# we now add some attention. This is done by using the average of the previous token values
# this is a simplification and much information is lost but better than bi-gram

class BigramLanguageModel_v2(nn.Module):
  def __init__(self, vocab_size, block_size, n_embed):
    super().__init__()

    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    self.lm_head = nn.Linear(n_embed, vocab_size)

  def forward(self, contexts, targets=None):
    B, T = contexts.shape

    tok_emb = self.token_embedding_table(contexts)
    pos_emb = self.position_embedding_table(torch.arange(T, device='cpu'))
    x = tok_emb + pos_emb

    logits = self.lm_head(x) # Batch Time Channel tensor (BTC)

    if targets is None:
      return logits, None
    
    # we have provided targets so compute the loss
    # we need to reshape the data for use in cross entropy
    B, T, C = logits.shape
    logits = logits.view(B * T, C)
    targets = targets.view(B * T)

    # compare to the target using neg log likeliehood loss
    loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of incdices into the current context
    for _ in range(max_new_tokens):
      idx_cond = idx[:, -block_size:]
      logits, loss = self.forward(idx_cond)
      logits = logits[:, -1, :] # pick the last element from the time dimension B, _T_, C
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1) # samples from the probabilities
      idx = torch.cat((idx, idx_next), dim=1) # add the prediction to the time dimension
    return idx


In [17]:
mv2 = BigramLanguageModel_v2(vocab_size, block_size, 32)
train_model(mv2, train_data, val_data)

step 0: train loss 4.4447, val loss 4.4538
step 200: train loss 3.0448, val loss 3.0806
step 400: train loss 2.7923, val loss 2.8138
step 600: train loss 2.6951, val loss 2.7175
step 800: train loss 2.6318, val loss 2.6422
step 1000: train loss 2.5898, val loss 2.6273
step 1200: train loss 2.5729, val loss 2.5932
step 1400: train loss 2.5491, val loss 2.5716
step 1600: train loss 2.5370, val loss 2.5650
step 1800: train loss 2.5245, val loss 2.5360
step 2000: train loss 2.5303, val loss 2.5394
step 2200: train loss 2.5161, val loss 2.5195
step 2400: train loss 2.5093, val loss 2.5247
step 2600: train loss 2.5017, val loss 2.5208
step 2800: train loss 2.4922, val loss 2.5177
step 3000: train loss 2.5029, val loss 2.5093
step 3200: train loss 2.4925, val loss 2.5099
step 3400: train loss 2.5032, val loss 2.5120
step 3600: train loss 2.4886, val loss 2.4972
step 3800: train loss 2.4917, val loss 2.5189
step 4000: train loss 2.4918, val loss 2.5171
step 4200: train loss 2.4862, val loss 2.

In [18]:
output = mv2.generate(idx = newline, max_new_tokens=200)
print(decode(output[0].tolist()))


LI s f ve and, he nse le d'dill we,

LARlaut yore s unakt he d, tou--we t o ibousry teslounane, m nyoupire, s srgo 's? d de hou thok n nt d.
ORGENraceve le stin.
Ans fo ar iteanfond:

Istee LI fr r, o
